In [1]:
%reload_ext autoreload
%autoreload 2
import sys, os
print(sys.executable)
module_path = os.path.abspath(os.path.join('..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)
# Verify kernal path
import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv
from datetime import datetime
load_dotenv()

/Users/pravachanpatra/Documents/PYTHON/AI_ML_DL/Stock_Price_Predictor/venv/bin/python


True

In [2]:
import src.config as config
import hopsworks

# connect to the project
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

# connect to the feature store
feature_store = project.get_feature_store()

# connect to the feature group
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION,
)


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/835736
2024-08-19 22:06:51,395 WARNING: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead

2024-08-19 22:06:51,396 WARNING: using legacy validation callback
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# Feature view is defined to read data from feature store, define how you want to fetch data from potentially many different feature groups and merge it to get the final dataset. 
try:
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
except:
    print("Feaeture view already existed. Skip creation")

feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

Feaeture view already existed. Skip creation


In [4]:
# feature-view-obj.func(): gets training data-frame from hopsworks
ts_prices, _ = feature_view.training_data(
    description="Time-series daily stock prices"
)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.65s) 
2024-08-19 22:07:01,979 WARNING: VersionWarning: Incremented version to `171`.



In [5]:
# sort the data fetched from feature-store and make sure it is sorted, the last row should be the current date because we are getting most recetn data with feature pipeline
ts_prices.sort_values(by=["datetime"], inplace=True)
ts_prices

,volume,vw_avr_price,open_price,close_price,high_price,low_price,num_transactions,datetime,sma_5,sma_20,ema_5,ema_20,vol_5,daily_return,price_diff,volume_sma_5,volume_change
341,60362338.0,165.8939,166.37,164.87,167.8100,164.2000,540017.0,2022-08-02 00:00:00+00:00,164.870,168.983000,164.870000,164.870000,0.000000,0.000000,0.00,60362338.0,0.000000
691,60362338.0,165.8939,166.37,164.87,167.8100,164.2000,540017.0,2022-08-03 00:00:00+00:00,164.870,168.983000,164.870000,164.870000,0.000000,0.000000,0.00,60362338.0,0.000000
363,60362338.0,165.8939,166.37,164.87,167.8100,164.2000,540017.0,2022-08-04 00:00:00+00:00,164.870,168.983000,164.870000,164.870000,0.000000,0.000000,0.00,60362338.0,0.000000
698,60362338.0,165.8939,166.37,164.87,167.8100,164.2000,540017.0,2022-08-05 00:00:00+00:00,164.870,168.983000,164.870000,164.870000,0.000000,0.000000,0.00,60362338.0,0.000000
352,60362338.0,165.8939,166.37,164.87,167.8100,164.2000,540017.0,2022-08-06 00:00:00+00:00,165.754,169.627000,164.870000,164.870000,1.948854,0.000000,0.00,62866611.4,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,42388970.0,225.4078,223.92,226.05,226.8271,223.6501,562568.0,2024-08-16 04:00:00+00:00,222.258,217.277333,222.668217,219.071721,3.318896,0.005918,1.33,41340047.2,-0.057212
755,42388970.0,225.4078,223.92,226.05,226.8271,223.6501,562568.0,2024-08-17 00:00:00+00:00,223.962,217.672500,223.795478,219.736319,2.322040,0.000000,0.00,42511207.4,0.000000
756,42388970.0,225.4078,223.92,226.05,226.8271,223.6501,562568.0,2024-08-18 00:00:00+00:00,224.918,218.063000,224.546986,220.337622,1.878209,0.000000,0.00,42532774.4,0.000000
758,42388970.0,225.4078,223.92,226.05,226.8271,223.6501,562568.0,2024-08-19 00:00:00+00:00,225.784,218.425500,225.047990,220.881658,0.594794,0.000000,0.00,42903435.6,0.000000


In [6]:
from src.data import transform_ts_data_into_features_target

n_previous_days = 12
step_size = 1
features, targets = transform_ts_data_into_features_target(ts_prices, n_previous_days, step_size)
X = features
Y = targets
print(f'Features shape: {features.shape}')  # Should be (number_of_examples, n_previous_days, n_features)
print(f'Targets shape: {targets.shape}')    # Should be (number_of_examples,)


Number of sequences/examples: 746
Last val: 173.19
Target val: 173.19
Features shape: (746, 13, 16)
Targets shape: (746,)


In [1]:
# from src.data import train_test_split_tabular

# # combine features/targets to create tabular data
# features_and_target = pd.DataFrame(features.copy())
# features_and_target["target_close_price_next_day"] = targets   # target column name
# print(f"{features_and_target.shape=}")
# features_and_target

In [2]:
# from sklearn.model_selection import train_test_split
# from src.data import train_test_split_tabular

# X_train, X_test, y_train, y_test = train_test_split_tabular(features_and_target)
# print(f"X_train: {X_train.shape}") # (examples-in-train, num-features)
# print(f"y_train: {y_train.shape}") # (examples-in-train,) 1D beceause only one target
# print(f"X_test: {X_test.shape}")   # (examples-in-test, num-features)
# print(f"y_test: {y_test.shape}")

In [7]:
# LIGHTGBM REGRESSOR MODEL
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


num_samples, sequence_length, num_features = X.shape
X_flattened = X.reshape(num_samples, sequence_length * num_features)

# Create the LightGBM Regressor model
lgb_model = lgb.LGBMRegressor(objective='regression', n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)

# Train the model
lgb_model.fit(X_flattened, Y)

# Predict on the test set
y_pred = lgb_model.predict(X_flattened)

# Calculate the mean squared error
mse = mean_squared_error(Y, y_pred)
print(f"Mean Squared Error: {mse}")

# Print the first few predictions and actual values
print("Predictions:", y_pred[:5])
print("Actual values:", Y[:5])

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003718 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 51566
[LightGBM] [Info] Number of data points in the train set: 746, number of used features: 208
[LightGBM] [Info] Start training from score 174.834799
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [8]:
# SAVE MODEL AS PICKLE LOCALLY
import joblib

model_path = f'../models/lgb_model3.pkl'
joblib.dump(lgb_model, model_path)


['../models/lgb_model3.pkl']

In [9]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

# before saving model in hopsworks need to define its schema
input_schema = Schema(X)
output_schema = Schema(Y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)


In [10]:
# Connect to the Hopsworks model registry
model_registry = project.get_model_registry()

# Register the model
model = model_registry.sklearn.create_model(
    name="lightgbm_model_stock_prices3",
    metrics={"MSE": mse},
    description="LightGBM model for predicting stock prices 3",
    input_example=X[0].reshape(1, -1),
    model_schema=model_schema
)

# save model to hopsworks model registry
model.save(model_path)

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/278746 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/3344 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/216 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/835736/models/lightgbm_model_stock_prices3/14


Model(name: 'lightgbm_model_stock_prices3', version: 14)